In [1]:
# Code Block 1

import tensorflow as tf
from keras.preprocessing import sequence
import keras
import os
import numpy as np

In [2]:
# Code Block 2

#Get data
path_to_file = "./SongLyrics.txt"
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

In [3]:
# Code Block 3

#Encode the text
vocab = sorted(set(text))

char2idx = {u: i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)


def text_to_int(text):
    return np.array([char2idx[c] for c in text])


text_as_int = text_to_int(text)

In [4]:
# Code Block 4

print(f'Text: {text[:200]}\nEncoded: {text_as_int[:200]}')

Text: 


The state of Man
And of machine
Streets of Japan
2517
2517

The fate of Man
And of machine
It all began
2517
2517




Salvation code
The salvation code
The salvation code

I hold on to the notion
T
Encoded: [ 0  0  0 37 52 49  1 63 64 45 64 49  1 59 50  1 31 45 58  0 19 58 48  1
 59 50  1 57 45 47 52 53 58 49  0 36 64 62 49 49 64 63  1 59 50  1 28 45
 60 45 58  0 12 13 11 14  0 12 13 11 14  0  0 37 52 49  1 50 45 64 49  1
 59 50  1 31 45 58  0 19 58 48  1 59 50  1 57 45 47 52 53 58 49  0 27 64
  1 45 56 56  1 46 49 51 45 58  0 12 13 11 14  0 12 13 11 14  0  0  0  0
  0 36 45 56 66 45 64 53 59 58  1 47 59 48 49  0 37 52 49  1 63 45 56 66
 45 64 53 59 58  1 47 59 48 49  0 37 52 49  1 63 45 56 66 45 64 53 59 58
  1 47 59 48 49  0  0 27  1 52 59 56 48  1 59 58  1 64 59  1 64 52 49  1
 58 59 64 53 59 58  0 37]


In [5]:
# Code Block 5

def int_to_text(ints):
    try:
        ints = ints.numpy()
    except:
        print('Except: Already a numpy array\n')
    return ''.join(idx2char[ints])


print(int_to_text(text_as_int))

Except: Already a numpy array




The state of Man
And of machine
Streets of Japan
2517
2517

The fate of Man
And of machine
It all began
2517
2517




Salvation code
The salvation code
The salvation code

I hold on to the notion
That I just wasn't born to die
Buried beneath the motion
Of life I never stop to question why
I've been dreaming of a savior
To pull me from this lowly place
She's analog and digital
Halo of light around her face

The past becoming clearer
I'm getting closer
And every day I'm nearer
To the salvation code

The salvation code
The salvation code
It's all clearer now
And I hear her now
And I'm nearer to
The salvation code
It's all clearer now
And I hear her now
And I'm nearer to
The salvation code

I'm still sticking to the notion
There's a bigger story left to tell
Sunk way down low in Tokyo
In the lowest level of this hell
Transmissions coming from my savior
Recieving in this lonely place
They're analog and digital
And they're guiding me through time and space



In [6]:
# Code Block 6

#Creating Training samples
seq_lenght = 100
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_lenght + 1, drop_remainder=True)

In [7]:
# Code Block 7

def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]

    return input_text, target_text


dataset = sequences.map(split_input_target)

In [8]:
# Code Block 8

BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)
EMBEDING_DIM = 256
RNN_UNITS = 1024

BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)


def build_model(vocab_size, embeding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embeding_dim, batch_input_shape=[batch_size, None]),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model


model = build_model(VOCAB_SIZE, EMBEDING_DIM, RNN_UNITS, BATCH_SIZE)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           18176     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 71)            72775     
                                                                 
Total params: 5,337,927
Trainable params: 5,337,927
Non-trainable params: 0
_________________________________________________________________


In [9]:
# Code Block 9

def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [10]:
# Code Block 10

model.compile(optimizer='adam', loss=loss)

In [11]:
# Code Block 11

checkpoint_dir = './LyricGeneratorCheckpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt_{epoch}')

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True,
    save_freq=50
)

history = model.fit(data, epochs=250, callbacks=[checkpoint_callback])

Epoch 1/250
7/7 [==============================] - 19s 2s/step - loss: 3.6715
Epoch 2/250
7/7 [==============================] - 15s 2s/step - loss: 3.2192
Epoch 3/250
7/7 [==============================] - 15s 2s/step - loss: 3.1614
Epoch 4/250
7/7 [==============================] - 16s 2s/step - loss: 3.1242
Epoch 5/250
7/7 [==============================] - 16s 2s/step - loss: 3.0723
Epoch 6/250
7/7 [==============================] - 16s 2s/step - loss: 3.0025
Epoch 7/250
7/7 [==============================] - 16s 2s/step - loss: 2.9019
Epoch 8/250
7/7 [==============================] - 16s 2s/step - loss: 2.7752
Epoch 9/250
7/7 [==============================] - 16s 2s/step - loss: 2.6500
Epoch 10/250
7/7 [==============================] - 15s 2s/step - loss: 2.5420
Epoch 11/250
7/7 [==============================] - 15s 2s/step - loss: 2.4585
Epoch 12/250
7/7 [==============================] - 15s 2s/step - loss: 2.3976
Epoch 13/250
7/7 [==============================] - 15s 2s/st

In [12]:
# Code Block 12

#LoadModel from lastest Checkpoint to train
model = build_model(VOCAB_SIZE, EMBEDING_DIM, RNN_UNITS, 64)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [13]:
# Code Block 13

#LoadModel from lastest Checkpoint to use

model = build_model(VOCAB_SIZE, EMBEDING_DIM, RNN_UNITS, 1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [14]:
# Code Block 14
def generate_text(model, start_string, num_generate=400, temperature=1):
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    model.reset_states()

    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return start_string + ''.join(text_generated)

In [15]:
# Code Block 15

start_str = str(input('Type a starting string: '))

lenght = input('Type song lenght, leave blank for default(400): ')

if not lenght or lenght == "":
    lenght = 400
else:
    lenght = int(lenght)

temp = input('Type temperature, leave blank for default(1.0): ')

if not temp or temp == "":
    temp = 1.0
else:
    temp = float(temp)

print(generate_text(model, start_str, lenght, temp))

Robin is happyes and light you'll see, you know where I will be
Everywhere you, everywhere you
Everywhere you go

The city lights, they bleed into the distance
Our minds and bodies bleeding into one
And if you look back you will see two sets of footprints
You're next to me, you are my Aphelion

From Tokyo to New York streets, on the face of the deep I'm not alone
In a world of the ashes
But searth to frillen by


In [16]:
# Code Block 16

print(generate_text(model, "monke", 1000, 1.3))

monker free the unrelys to lese empty streets alone
I walk these empty streets alone
I walk these empty streets alone
I walk these empty streets alone
I walk these empty streets alone
I walk these empty streets aling
're u praying)
Hell calls my name as the codear frer
And not
And all intwa, whorao scarlowin to the notioned
Frow I cayn mind

Alorgest eden
"And arous deon sht
Whoa-oh!
Go from hiding
We synchronize
We are the (future bloodline)

Manufactured masses, a prosty mind
Choug, everywhing sobesare

Ih strteo this waking dream
As darkness fallqing
Our my
And bred ellore tonight
(Tonight, the unknown
I'm rel you (ah-ohLide

In my merom eyes
You know it's thriller, thrise int'r taken
Coullone)
(In a world of color less on fad alone Gorour thoughts are naked tonight
In this neon glow
Lit up like a firefore you moonlight
You see a sights that bring you down so low
A light shines in the darkness, purified

You and I
Sanctified
In the night
(You and I) You and, (you're next to me, eper